In [1]:
import pandas as pd
import numpy as np
import re
import regex
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import nltk
from sklearn.feature_extraction.text import  TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [105]:
from Carga_dataset import *

In [107]:
#Convertimos la columna en string para poder trabajar con el texto
df['Descripcion limpia']= df['descripcion'].astype(str)
df['Descripcion limpia']

id_producto
2792-88-LE21-1        CÁMARA RETINAL NO MIDRIÁTICA - ESPECIFICACIONE...
1704-52-LP21-1        SERVICIO DE MUDANZA DE ENSERES PROPIOS Y EN US...
1003-20-L121-1                                    LEÑA SECA CERTIFICADA
4236-21-L121-1                             MARCADOR BLANCO PARA METALES
4236-21-L121-2                                                  PLOMADA
                                            ...                        
1058141-35-LQ21-1              ALTEPLASA FA 50 MG CAJA X 2 FA [216P008]
1058141-35-LQ21-4           TENECTEPLASA FA 10.000 UI (50 MG) [216P185]
1173455-12-LE21-41               manguera transparente para nivel (mts)
1057461-64-LE21-17                        CINTA ADHESIVA 18 MM X 30 MT 
4309-381-LQ21-1       10-100-214-208-01 Olanzapina tableta liofiliza...
Name: Descripcion limpia, Length: 54849, dtype: object

In [108]:
#Removemos la palabra "no" de nuestro stopword ya que nos puede ser útil
sw = stopwords.words('spanish')
sw.remove("no")

In [109]:
#Definimos nuestra función para quitar las tildes
def sin_tildes(s):
    tildes = (
        ("á", "a"),
        ("é", "e"),
        ("í", "i"),
        ("ó", "o"),
        ("ú", "u"),
    )
    for origen, destino in tildes:
        s = s.replace(origen, destino)
    return s




#Definimos una función para el preprocesamiento de texto

def texto_limpio(texto):
    texto = texto.lower() # convertir en minúsculas
    texto = re.sub(r"[\W]+", " ",texto) # remover caract especiales
    texto = sin_tildes(texto) # remove tildes
    texto = texto.split() # tokenizar
    texto = [palabra for palabra in texto if palabra not in sw] # stopwords
    texto = " ".join(texto)
    return texto

In [110]:
df['Descripcion limpia'] = df['Descripcion limpia'].apply(lambda texto: texto_limpio(texto)) #Aplicamos la función texto_limpio para limpiar las descripciones
df.head()

,id_licitacion,id_producto,Rubro2,Rubro3,nombre_producto,descripcion,label,Rubro1_Artículos de electrónica,Rubro1_Artículos de fabricación y producción,Rubro1_Artículos eléctricos y de iluminación,...,"Rubro1_Servicios de perforación de minería, petróleo y gas",Rubro1_Servicios de producción y fabricación industrial,"Rubro1_Servicios de transporte, almacenaje y correo","Rubro1_Servicios editoriales, de diseño, publicidad, gráficos y artistas","Rubro1_Servicios financieros, pensiones y seguros",Rubro1_Servicios medioambientales,"Rubro1_Servicios profesionales, administrativos y consultorías de gestión empresarial","Rubro1_Tecnologías de la información, telecomunicaciones y radiodifusión",Rubro1_Vehículos y equipamiento en general,Descripcion limpia
id_producto,,,,,,,,,,,,,,,,,,,,,
2792-88-LE21-1,2792-88-LE21,2792-88-LE21-1,Productos quirúrgicos,"Equipo quirúrgico, accesorios y productos rela...",Equipo quirúrgico de fragmatome retinal vítreo...,CÁMARA RETINAL NO MIDRIÁTICA - ESPECIFICACIONE...,0,0,0,0,...,0,0,0,0,0,0,0,0,0,camara retinal midriatica especificaciones bas...
1704-52-LP21-1,1704-52-LP21,1704-52-LP21-1,Transporte de correo y carga,Transporte de carga por carretera,Servicios de transporte local en camión,SERVICIO DE MUDANZA DE ENSERES PROPIOS Y EN US...,0,0,0,0,...,0,0,1,0,0,0,0,0,0,servicio mudanza enseres propios uso hospital ...
1003-20-L121-1,1003-20-L121,1003-20-L121-1,Chatarra y desechos no metálicos,Desechos no metálicos,Desperdicios o restos de madera,LEÑA SECA CERTIFICADA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,leña seca certificada
4236-21-L121-1,4236-21-L121,4236-21-L121-1,Suministros de oficina,Instrumentos de escritura,Marcadores,MARCADOR BLANCO PARA METALES,0,0,0,0,...,0,0,0,0,0,0,0,0,0,marcador blanco metales
4236-21-L121-2,4236-21-L121,4236-21-L121-2,Herramientas manuales,Herramientas de medición y trazado,Péndulos,PLOMADA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,plomada


In [111]:
stemmer=SnowballStemmer("spanish")
#Obtención de texto raíz limpio
def texto_raiz(texto):    
    texto = texto.lower() # convertir en minúsculas
    texto = re.sub(r"[\W]+", " ",texto) # remover caract especiales
    texto = sin_tildes(texto) # remove tildes
    texto = texto.split() # tokenizar
    texto = [palabra for palabra in texto if palabra not in sw] # stopwords
    texto = [stemmer.stem(palabra) for palabra in texto] #stem
    texto = " ".join(texto)
    
    return texto

In [112]:
df['Descripcion raiz limpia']= df['Descripcion limpia'].apply(lambda texto: texto_raiz(texto)) #Aplicamos la función texto_raiz que nos convierte las palabras en sus raíces
df.head()

,id_licitacion,id_producto,Rubro2,Rubro3,nombre_producto,descripcion,label,Rubro1_Artículos de electrónica,Rubro1_Artículos de fabricación y producción,Rubro1_Artículos eléctricos y de iluminación,...,Rubro1_Servicios de producción y fabricación industrial,"Rubro1_Servicios de transporte, almacenaje y correo","Rubro1_Servicios editoriales, de diseño, publicidad, gráficos y artistas","Rubro1_Servicios financieros, pensiones y seguros",Rubro1_Servicios medioambientales,"Rubro1_Servicios profesionales, administrativos y consultorías de gestión empresarial","Rubro1_Tecnologías de la información, telecomunicaciones y radiodifusión",Rubro1_Vehículos y equipamiento en general,Descripcion limpia,Descripcion raiz limpia
id_producto,,,,,,,,,,,,,,,,,,,,,
2792-88-LE21-1,2792-88-LE21,2792-88-LE21-1,Productos quirúrgicos,"Equipo quirúrgico, accesorios y productos rela...",Equipo quirúrgico de fragmatome retinal vítreo...,CÁMARA RETINAL NO MIDRIÁTICA - ESPECIFICACIONE...,0,0,0,0,...,0,0,0,0,0,0,0,0,camara retinal midriatica especificaciones bas...,cam retinal midriat especif bas tecnic
1704-52-LP21-1,1704-52-LP21,1704-52-LP21-1,Transporte de correo y carga,Transporte de carga por carretera,Servicios de transporte local en camión,SERVICIO DE MUDANZA DE ENSERES PROPIOS Y EN US...,0,0,0,0,...,0,1,0,0,0,0,0,0,servicio mudanza enseres propios uso hospital ...,servici mudanz enser propi uso hospital san ma...
1003-20-L121-1,1003-20-L121,1003-20-L121-1,Chatarra y desechos no metálicos,Desechos no metálicos,Desperdicios o restos de madera,LEÑA SECA CERTIFICADA,0,0,0,0,...,0,0,0,0,0,0,0,0,leña seca certificada,leñ sec certific
4236-21-L121-1,4236-21-L121,4236-21-L121-1,Suministros de oficina,Instrumentos de escritura,Marcadores,MARCADOR BLANCO PARA METALES,0,0,0,0,...,0,0,0,0,0,0,0,0,marcador blanco metales,marcador blanc metal
4236-21-L121-2,4236-21-L121,4236-21-L121-2,Herramientas manuales,Herramientas de medición y trazado,Péndulos,PLOMADA,0,0,0,0,...,0,0,0,0,0,0,0,0,plomada,plom


In [ ]:
distribucion=nltk.FreqDist(count.vocabulary_) 
lista_frecuencias=distribucion.most_common() #Para ordenar la distribución de la frecuencia de palabras
print(lista_frecuencias)

In [78]:
#Creamos nuestra bolsa de palabras

count = CountVectorizer()
descripcion = np.array(df['Descripcion limpia'])
bag = count.fit_transform(descripcion)

In [79]:
#Observamos las palabras presentadas en las descripciones limpias
n_palabras= count.vocabulary_
n_palabras

{'camara': 13628,
 'retinal': 33764,
 'no': 28945,
 'midriatica': 27673,
 'especificaciones': 19832,
 'bases': 12262,
 'tecnicas': 36707,
 'servicio': 34984,
 'mudanza': 28294,
 'enseres': 19406,
 'propios': 32273,
 'uso': 38353,
 'hospital': 23345,
 'san': 34499,
 'martin': 26928,
 'quillota': 32776,
 'ofertar': 29349,
 'segun': 34767,
 'anexo': 10679,
 'leña': 25753,
 'seca': 34699,
 'certificada': 14487,
 'marcador': 26835,
 'blanco': 12756,
 'metales': 27412,
 'plomada': 31333,
 'azadon': 11941,
 'mango': 26720,
 'escobillon': 19695,
 'aseo': 11440,
 'municipal': 28430,
 'caja': 13489,
 'lapiz': 25440,
 'carpintero': 14019,
 'hojas': 23268,
 'marcosierra': 26855,
 'cinta': 14891,
 'papel': 30162,
 'equivalente': 19587,
 'masking': 26959,
 'tape': 36593,
 'pegote': 30550,
 'ancho': 10621,
 '48': 6149,
 '40': 5601,
 'mm': 27867,
 'carpeta': 14012,
 'carton': 14064,
 'pigmentadas': 31022,
 'oficio': 29366,
 'color': 15534,
 'azul': 11969,
 'plastificada': 31264,
 'accoclip': 9365,
 'a

In [94]:
vectorizador = TfidfVectorizer(max_features=20)
 
#Crear una matriz dispersa
cantidades = count.fit_transform(descripcion)
valores = vectorizador.fit_transform(descripcion)
 
print ("--- Palabras ---")
print(vectorizador.get_feature_names())
print ("... Vector de cantidades ...")
print (cantidades)



--- Palabras ---
['10', '100', '24', '30', 'bases', 'certificado', 'cm', 'comprimidos', 'menos', 'meses', 'mg', 'ml', 'precio', 'presentacion', 'publico', 'punto', 'segun', 'similar', 'tecnicas', 'vista']
... Vector de cantidades ...
  (0, 13628)	1
  (0, 33764)	1
  (0, 28945)	1
  (0, 27673)	1
  (0, 19832)	1
  (0, 12262)	1
  (0, 36707)	1
  (1, 34984)	1
  (1, 28294)	1
  (1, 19406)	1
  (1, 32273)	1
  (1, 38353)	1
  (1, 23345)	1
  (1, 34499)	1
  (1, 26928)	1
  (1, 32776)	1
  (1, 29349)	1
  (1, 34767)	1
  (1, 10679)	1
  (2, 25753)	1
  (2, 34699)	1
  (2, 14487)	1
  (3, 26835)	1
  (3, 12756)	1
  (3, 27412)	1
  :	:
  (54845, 38107)	1
  (54845, 36832)	1
  (54845, 3854)	1
  (54846, 28278)	1
  (54846, 37612)	1
  (54846, 26722)	1
  (54846, 28926)	1
  (54847, 14891)	1
  (54847, 27867)	1
  (54847, 9647)	1
  (54847, 2916)	1
  (54847, 4709)	1
  (54847, 28267)	1
  (54848, 1139)	1
  (54848, 1138)	2
  (54848, 27521)	1
  (54848, 29425)	1
  (54848, 29599)	1
  (54848, 194)	1
  (54848, 36463)	1
  (54848, 181

In [93]:
print ("¨¨¨ Vector de valores ¨¨")
print (valores)

¨¨¨ Vector de valores ¨¨
  (0, 18)	0.6988178154444844
  (0, 4)	0.7152996999981185
  (1, 16)	1.0
  (24, 1)	1.0
  (25, 1)	1.0
  (26, 16)	0.6896126129074674
  (26, 18)	0.7241784615127238
  (28, 16)	0.6896126129074674
  (28, 18)	0.7241784615127238
  (29, 16)	0.6896126129074674
  (29, 18)	0.7241784615127238
  (51, 6)	1.0
  (55, 2)	0.45218978454882747
  (55, 17)	0.46670806387695213
  (55, 16)	0.42108387874000597
  (55, 18)	0.44219010755632
  (55, 4)	0.45261932979769437
  (56, 11)	1.0
  (57, 11)	1.0
  (58, 11)	1.0
  (59, 11)	1.0
  (60, 11)	1.0
  (61, 8)	1.0
  (62, 4)	1.0
  (63, 4)	1.0
  :	:
  (54825, 7)	0.8097379922809425
  (54825, 10)	0.5867916017265655
  (54827, 7)	0.6105485042558791
  (54827, 10)	0.4424452577986046
  (54827, 3)	0.6568658293764676
  (54828, 7)	0.8097379922809425
  (54828, 10)	0.5867916017265655
  (54829, 7)	0.8097379922809425
  (54829, 10)	0.5867916017265655
  (54833, 7)	0.6808112827891167
  (54833, 3)	0.7324588706726387
  (54834, 11)	1.0
  (54835, 10)	1.0
  (54836, 10)	1.0